# function to solve subsistence problem

In [25]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

# setup 

In [26]:
#!git reset --hard origin/master  # To revert to original
!pip install -r requirements.txt --upgrade

In [21]:
api_lst = ["9pWr4Z38nFyytVVF2LPQWJ5G3x5SrVO0Px3MbQOl",
           "bShyCnUYYfY6eP3qyUt6VGY1MqEQklrOeuwPUps5",
           "5EVJteEMrjnX73GRj4Q9qqB3NWqIObFujtP4fBGI",
           "saSE5uqp940Hmw6imfc1FaCo6qFJS9dOH8RIjSJW"
          ]
apikey = api_lst[3]

# input data

# compile data on food prices 

In [27]:
import pandas as pd
from eep153_tools.sheets import read_sheets

df = read_sheets("https://docs.google.com/spreadsheets/d/1Ml9KIH_HLl64Y8zTldoghUu5ZhTY7IXanN77itVAxRM/edit#gid=0", 
                sheet='prices')
df

Key available for students@eep153.iam.gserviceaccount.com.


,Food,Quantity,Units,Price,Date,Location,FDC
0,Bread,1.0,serving,1.33,2023/02/28,MI Ranchito Bayside Market,2431121
1,Cheese,8.0,oz,3.49,2023/02/27,Trader Joe's,2383425
2,Ground Beef,16.0,oz,4.99,2023/02/27,Trader Joe's,1662368
3,cheese burger,1.0,cheese burger,3.01,2023/02/27,Trader Joe's,2022263
4,1% fat milk,1.0,gallon,4.39,2023/02/27,Trader Joe's,2340764
5,red apple,1.0,pound,0.99,2023/02/28,MI Ranchito Bayside Market,1750339
6,edamame,1.0,pound,1.89,2023/02/28,MI Ranchito Bayside Market,2384458
7,mini carrot,1.0,pound,1.49,2023/02/28,Target,2345173
8,mandarin,2.0,pound,3.99,2023/02/27,Trader Joe's,169105
9,cauliflower,2.0,pound,2.99,2023/02/27,Trader Joe's,2409200


In [28]:
# import fooddatacentral as fdc
# import warnings

# D = {}
# count = 0
# for food in  df.Food.tolist():
#     try:
#         FDC = df.loc[df.Food==food,:].FDC[count]
#         count+=1
#         D[food] = fdc.nutrients(apikey,FDC).Quantity
#     except AttributeError: 
#         warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

# FoodNutrients = pd.DataFrame(D,dtype=float)

/tmp/ipykernel_134/1532008836.py:12: UserWarning: Couldn't find FDC Code Bread for food 2431121.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/tmp/ipykernel_134/1532008836.py:12: UserWarning: Couldn't find FDC Code Cheese for food 2383425.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/tmp/ipykernel_134/1532008836.py:12: UserWarning: Couldn't find FDC Code Ground Beef for food 1662368.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/tmp/ipykernel_134/1532008836.py:12: UserWarning: Couldn't find FDC Code cheese burger  for food 2022263.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/tmp/ipykernel_134/1532008836.py:12: UserWarning: Couldn't find FDC Code 1% fat milk for food 2340764.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/tmp/ipykernel_134/1532008836.py:12: UserWarning: Couldn't find FDC Code edamame for food 2384458.
  warnings.warn("Couldn't find FDC Code

In [29]:
FoodNutrients = pd.read_csv("FoodNutrients.csv")
FoodNutrients

FileNotFoundError: [Errno 2] No such file or directory: 'FoodNutrients.csv'